# Data Analysis Notebook

This notebook contains code for an interactive GUI that can run the lick detection algorithm and display results. It is based on the MATLAB lickDetector.m script, and all code for the analysis algorithms is contained in data_analysis.py.

In [1]:
import h5py
import numpy as np
import pandas as pd
from data_analysis import filter_data

In [2]:
# TODO: Create file picker GUI here to allow selection of multiple data files
raw_data_filename = "raw_data_2025-05-20_13-28-35.h5"
filtered_data_filename = "filtered" + raw_data_filename[3:]

In [3]:
# Will be populated by the file picker later
data_files = [raw_data_filename]

In [4]:
# TODO: File picker GUI for selecting the animal-sensor map file
layout = pd.read_csv("layouts/default_layout.csv", header=None, index_col=0)

In [6]:
# Loop through data files and run the filter_data script on them
for raw_data in data_files:
    print(f"Working on file: {raw_data}")
    with h5py.File(raw_data_filename, 'r') as raw_h5, h5py.File(filtered_data_filename, 'w') as filtered_h5:
        filter_data(raw_h5, filtered_h5, layout, raw_data_filename[:-3] + '.log')

Working on file: raw_data_2025-05-20_13-28-35.h5
Animal ID A1
Animal ID A2
Animal ID A3
Animal ID A4
Animal ID A5
Animal ID A6
Animal ID A7
Animal ID A8
Caught KeyError 'consumed_vol', volumes were likely not recorded for A8
Caught KeyError 'weight', weight was likely not recorded for A8
Animal ID A9
Caught KeyError 'consumed_vol', volumes were likely not recorded for A9
Caught KeyError 'weight', weight was likely not recorded for A9
Animal ID A10
Caught KeyError 'consumed_vol', volumes were likely not recorded for A10
Caught KeyError 'weight', weight was likely not recorded for A10
Animal ID A11
Caught KeyError 'consumed_vol', volumes were likely not recorded for A11
Caught KeyError 'weight', weight was likely not recorded for A11
Animal ID A12
Caught KeyError 'consumed_vol', volumes were likely not recorded for A12
Caught KeyError 'weight', weight was likely not recorded for A12
Animal ID A13
Caught KeyError 'consumed_vol', volumes were likely not recorded for A13
Caught KeyError 'we